# Convolutional Auto-Encoder

In [8]:
# Imports
from convvae import ConvVae
import numpy as np
import PIL
from sklearn.model_selection import train_test_split

In [ ]:
import Neurosmash

# These are the default environment arguments. They must be the same as the values that are set in the environment GUI.
ip         = "127.0.0.1" # Ip address that the TCP/IP interface listens to (127.0.0.1 by default)
port       = 13000       # Port number that the TCP/IP interface listens to (13000 by default)

# This is the size of the texture that the environment is rendered.
# This is set to 784 by default, which will result in a crisp image but slow speed.
# You can change the size to a value that works well for your environment but should not go too low.
size       = 64# 96, 192

# This is the simulation speed of the environment. This is set to 1 by default.
# Setting it to n will make the simulation n times faster.
# In other words, less (if n < 1) or more (if n > 1) simulation time will pass per step.
# You might want to increase this value to around 10 if you cannot train your models fast enough
# so that they can sample more states in a shorter number of steps at the expense of precision.
timescale  = 5

# This is an example agent.
# It has a step function, which gets reward/state as arguments and returns an action.
# Right now, it always outputs a random action (3) regardless of reward/state.
# The real agent should output one of the following three actions:
# none (0), left (1) and right (2)
agent = Neurosmash.Agent() 

# This is the main environment.
# It has a reset function, which is used to reset the environment before episodes.
# It also has a step function, which is used to which steps one time point
# It gets an action (as defined above) as input and outputs the following:
# end (true if the episode has ended, false otherwise)
# reward (10 if won, 0 otherwise)
# state (flattened size x size x 3 vector of pixel values)
# The state can be converted into an image as follows:
# image = np.array(state, "uint8").reshape(size, size, 3)
# You can also use to Neurosmash.Environment.state2image(state) function which returns
# the state as a PIL image
environment = Neurosmash.Environment(ip, port, size, timescale) 



In [ ]:
# The following steps through an entire episode from start to finish with random actions (by default)

end, reward, state = environment.reset()
nr_images = 1000
shape = np.array(state).reshape((size,size,3)).shape
images = np.zeros((nr_images, shape[0], shape[1], shape[2]))

i = 0
while i < nr_images:
    end, reward, state = environment.reset()
    while (end == 0):
        action = agent.step(end, reward, state)
        end, reward, state = environment.step(action)
        images[i] = np.array(state).reshape(shape)
        i += 1
        if i >= nr_images:
            end = 1

    # Let's run it a few more steps so that the things have time to settle down
    for _ in range(100):
        if i >= nr_images:
            continue
        action = agent.step(end, reward, state)
        end, reward, state = environment.step(action)
        images[i] = np.array(state).reshape(shape)
        i += 1

environment.state2image(state)

In [ ]:
train_iter = mx.io.NDArrayIter(data={'data': train_data}, batch_size = batch_size)
test_iter = mx.io.NDArrayIter(data={'data': test_data}, batch_size = batch_size)

vae = ConvVae()
vae.initialize()
vae.hybridize()
trainer = gluon.Trainer(net.collect_params(), 'adam', {'learning_rate': .001})

In [ ]:
def train_vae(n_epochs):
    train_loss = []
    valid_loss = []

    for epoch in n_epochs:
        